# MapReduce
## El nacimiento del BigData
---


Según un análisis rápido de la popularidad de términos de busqueda en google terms, es a partir de 2012 cuando explota la popularidad del termino **Big Data**. A pesar de que es difícil explicar toda la casuística que pudo desencadenar la viralización del termino, es bastante probable que fuese iniciada por su mención especial por parte del World Economy Forum en su informe [Big Data, Big Impact: New Possibilities for International Development](http://www3.weforum.org/docs/WEF_TC_MFS_BigDataBigImpact_Briefing_2012.pdf) y su posterior difusión gracias a un artículo del New York Times con el suculento título [The Age of Big Data](http://www.nytimes.com/2012/02/12/sunday-review/big-datas-impact-in-the-world.html).

<img src='img/data_terms_trends.png' width=400>
$$\text{Fig.1 Importancia relativa de términos de busqueda en google relacionados con el mundo del dato}$$

No obstante, en contra de lo que parece indicar la gráfica anterior, deberíamos situar el nacimiento del Big Data un poco antes, concretamente en 2004, cuando J. Dean y S. Ghemawat, dos ingenieros de Google, publican un artículo titulado [MapReduce: Simplified Data Processing on Large Clusters](https://static.googleusercontent.com/media/research.google.com/en//archive/mapreduce-osdi04.pdf).

En este artículo, Jeffrey Sanjay exponen de forma clara un nuevo modelo de programación que reduce a dos funciones básicas el procesado de un fichero. Estás dos funciones, *Map* y *Reduce*, inspiradas en las funciones primitivas homónimas de Lisp y otros lenguajes funcionales, consisten básicamente en contar (map) cuantas ocurrencias hay de cada tipo de elemento (por ejemplo: número de veces que se repite una palabra en un texto) y posteriormente sumar (reducir) el número de elementos que hay de cada tipo.

A pesar de su sencillez, este paradigma resulta ser muy versatil ya que muchas de las tareas a realizar en grandes volumenes de datos se pueden expresar bajo este modelo.

<img src='img/map-reduce-execution.png'>

A pesar de su sencillez, este paradigma resulta ser muy versatil ya que muchas de las tareas a realizar en grandes volumenes de datos se pueden expresar bajo este modelo. La gracia del concepto MapReduce es que es facilmente paralelizable y permite a usuarios sin experiencia en computación distribuida usar los recursos de grandes *clusters* (conjuntos de máquinas).

---

## Map(y)Reduce: una versión simplificada en python
Adaptada de Data Science from scratch.

Como punto de partida, vamos a crear una función que cuente las palabras de una serie de documentos.

Cargar los docs desde github

In [ ]:
import requests

path_othello = 'https://raw.githubusercontent.com/juanhuguetgarcia/MADM-2017/master/4-BigData/data/othello.txt'
path_mcbeth = 'https://raw.githubusercontent.com/juanhuguetgarcia/MADM-2017/master/4-BigData/data/mcbeth.txt'

paths = [path_othello, path_mcbeth]

docs = []
for path in paths:
    data = requests.get(path)
    docs.append(data.text)

O bien, desde local...

In [ ]:
local_paths = ['data/othello.txt', 'data/mcbeth.txt']

docs = []
for path in local_paths:
    with open(path, 'r') as f:
        docs.append(f.read())

Crea una funcion `tokenize` que dada una frase como cadena de caracteres devuelva una lista con las palabras en minúsculas

In [ ]:
def tokenize(mensaje):
    """
    Dado un mensaje, es decir, una serie de palabras en forma de cadena realiza:
    1. Convierte a minusculas
    2. Extrae las palabras usando una expresión regular
    """
    mensaje = ...
    palabras = ...
    return palabras

### Método clásico: Crea una función que cuente todas las palabras de los documentos de forma secuencial

In [ ]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

def cuenta_palabras_secuencial(documents, stop_words={}):
    """
    Cuenta las palabras que existen en una serie de documentos.
    """
    # Inicializa un diccionario vacio. En el pondremos la palabras y el número
    # de veces que aparecen en forma de clave, valor
    wordcount = {}
    
    # Crea un loop que recorra los documentos.
    
        # tokeniza el documento
    
        #crea un loop que recorra las palabras del documento
        for ... in ...:
    
            if word not in stop_words:
                # Comprueba si la palabra existe ya en el diccionario inicial.
                # Si no existe, asignale al diccionario una nueva clave con el nombre de la
                # palabra y dale el valor 1.
                # En caso contrario, actualiza el valor que contiene y súmale uno.
                if word not in wordcount:
                    ...
                else:
                    ...
            else:
                continue
    return

Vamos a ver si funciona...

In [ ]:
words_seq = ...

sorted_words_seq = sorted(words_seq.items(), key=lambda x: x[1], reverse=True)

sorted_words_seq[:5]

En el snippet de código anterior vemos que crear un contador de palabras no es complicado. Sin embargo, esta aproximación tiene dos inconvenientes mayores:

1. Al ser secuencial, solo puede procesar un documento a la vez
1. La máquina que hace el procesado tiene que tener el documento en memoria

Sin embargo, si tuviesemos acceso a 1000 máquinas, podríamos utilizar el esquema Map-Reduce de forma que:

1. Repartir los documentos entre las distintas máquinas
1. Que cada máquina **mapee** los documentos generando montones de pares clave-valor
1. Que un colector recoja los pares clave-valor y los envíe a un **reductor** de forma que cada clave única este en un sólo colector
1. Que cada **reductor agrupe** todos los pares clave-valor **por clave y reduzca** (sume) todos los valores

Vamos a definir las funciones de mapeo y reducción que nos permitirían, dada la infraestructura, hacer un conteo de palabras de forma distribuída.


In [ ]:
## funcion de mapeo

def word_count_mapper(document, stop_words={}):
    """
    Para cada una de las palabras del documento, emite una tupla ('palabra', 1)
    """
    ## Inicializa una lista vacia
    word_counter = []
    
    # haz un loop para cada una de las palabras tokenizadas del documento y 
    # si no está en la lista negra, haz un append a la lista vacia con una tupla
    # (word, 1)
    for ... in ... :
        if ... not in ...:
            word_counter.append(...)
    return ...

In [ ]:
## funcion de reducción

def word_count_reducer(word_counter):
    """
    Dada una tupla (word, lista_con_unos), devuelve una tupla con la palabra
    y la suma de apariciones
    """
    word = ...
    count_list = ...
    return ...

In [ ]:
## poniendolo todo junto...

from collections import defaultdict

def cuenta_palabras_mapreduce(documents, stop_words={}):
    """
    Cuenta las palabras de los documentos usando las funciones de wc_mapper y wc_reducer (MapReduce)
    """
    collector = defaultdict(list)
    
    ## Map
    for ... in ...:
        for ..., ... in ...:
            ...
    
    ## Reduce
    reduced_words = []
    for ... in collector.items():
        ...
    return reduced_words

In [ ]:
words = ...

sorted_words = sorted(words, key=lambda x: x[1], reverse=True)

sorted_words[:5]